Dosyaların okunması

In [1]:
# logoları crop edip dosya dosya ayıracak

In [ ]:
# 1
# data klasörünü oku (train için kullandığın dosya)
# resim isimlerini al os.listdir()
# logo_uuid.png # böl
# logoya göre böl # dict içerisine logo: ["logo_uuid.png", "logo_uuid.png", "logo_uuid.png"]

# !!json var ise oku, json içerisindeki logoları ele!!

# resim okuman grecek
# modele sor
# modelden çıkanları cropla bir listeye at
# crop edilen alanın boyutuna
# 20 tane logon varsa hepsinin boyutuna sahipsin -> 30x40 32x43, bu boyutların hepsinin ortalamasını alırsın
# sonra resimleri bu ortalama göre resize edersin!
# aldığın ortalamayı da bir json içerisine kaydedersin

# api -> akbank -> size: 80,20 -> dışarıdan akbank fotoğrafı geldi resize 80,20 -> feature matching yap




# 
# klasörü de sen yaratıcaksın kodla ve sınıf isminda olacak
# onlarıda sırayla indexine göre logo adı ile beraber akbank_0 akbank_1 olarak akbank klasörüne kaydet


In [ ]:
def cropped_into_dict():   

In [ ]:
def crooped_average():
    
    resimlleri classlarla okuyup class ort crop value bul

    def ort_class_crop()

    
    def reshape all class by crooped_average


    class icinde yuzde kac similarity var bunu bul 
    


In [1]:
import cv2
import os 
import yaml

def crop_logos():

    with open("dataset/data.yaml", 'r') as file:
        # load the contents of the file into a Python object
        data = yaml.load(file, Loader=yaml.FullLoader)

    # extract the names from the data
    class_yaml = data["names"]


    img_folder_path = "dataset/train/images/"
    txt_folder_path = "dataset/train/labels/"
    cropped_folder_path = "cropped/"
    
    class_names = class_yaml
    counter = 0

    # loop through all the images in the image folder
    for img_name in os.listdir(img_folder_path):
        counter+=1
        print(counter)
        # construct the full path to the image and txt file
        img_path = os.path.join(img_folder_path, img_name)
        txt_path = os.path.join(txt_folder_path, img_name.replace('.jpg', '.txt'))
        
        # read the image and the txt file
        img = cv2.imread(img_path)
        try:
            with open(txt_path) as f:
                lines = f.read().strip().split('\n')
            
            for line in lines:
                # split the line by space to separate class name and coordinates
                class_name, x, y, w, h = line.strip().split()
                
                label_name = class_names[int(class_name)]
                x, y, w, h = float(x), float(y), float(w), float(h)
                x1, y1, x2, y2 = int((x-w/2) * 1280), int((y-h/2) * 720), int((x+w/2) * 1280), int((y+h/2) * 720)

                #print(x1,y1,x2,y2)
                # crop the image using the coordinates
                cropped_img = img[y1:y2, x1:x2]

                # construct the full path to the save the cropped image
                cropped_img_path = os.path.join(cropped_folder_path, f"{label_name}_{counter}.jpg")

                # save the cropped image
                cv2.imwrite(cropped_img_path, cropped_img)

        except ValueError:
            print("value error E$$EQ")
            continue

In [2]:
crop_logos()

1


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/labels/images/a101_000_png.rf.7aa86e355974cba4057e3fae6d210701.txt'